In [53]:
window_size = [7,14,30,60]
num_features = 8 #depends on how much features we have
output_days = 7
output_features = 2 #min and max

Base Model building

In [54]:
import tensorflow as tf
from tensorflow.keras import layers,models

In [135]:
#loss function definition
def time_weighted_loss(y_true, y_pred, weights, lambda_=1.0, gamma=0.1):
    """
    Time-weighted loss function for 7-day min and max price prediction.
    
    Args:
        y_true (tf.Tensor): True values (concatenated min and max), shape (batch_size, 14).
        y_pred (tf.Tensor): Predicted values (concatenated min and max), shape (batch_size, 14).
        weights (list or tf.Tensor): Time-based weights for each day, shape (7,).
        lambda_ (float): Weight for the min-max constraint penalty.
        gamma (float): Weight for the temporal smoothness penalty.
    
    Returns:
        tf.Tensor: Total loss value.
    """
    # Split y_true and y_pred into min and max
    y_true_min, y_true_max = y_true[:, 7:], y_true[:, :7]
    y_pred_min, y_pred_max = y_pred[:, 7:], y_pred[:, :7]
    
    # Ensure weights are a tensor
    weights = tf.convert_to_tensor(weights, dtype=tf.float32)
    
    # Time-weighted MSE for min and max
    mse_min = tf.reduce_mean(weights * tf.square(y_true_min - y_pred_min))
    mse_max = tf.reduce_mean(weights * tf.square(y_true_max - y_pred_max))
    
    # Time-weighted min-max constraint penalty
    penalty = tf.reduce_sum(weights * tf.square(tf.maximum(0.0, y_pred_min - y_pred_max)))
    
    # Time-weighted temporal smoothness (optional)
    smoothness_min = tf.reduce_sum(weights[1:] * tf.square(y_pred_min[:, 1:] - y_pred_min[:, :-1]))
    smoothness_max = tf.reduce_sum(weights[1:] * tf.square(y_pred_max[:, 1:] - y_pred_max[:, :-1]))
    
    # Total loss
    total_loss = (mse_min + mse_max) + lambda_ * penalty + gamma * (smoothness_min + smoothness_max)
    return total_loss

In [136]:
from functools import partial

# Define weights (e.g., linear decay)
weights = [1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4]

# Wrap the loss function with fixed arguments
custom_loss = partial(time_weighted_loss, weights=weights, lambda_=1.0, gamma=0.1)

In [55]:
def baseline_LSTM_model(window_size, num_features, output_days, output_features):
    inputs = layers.Input(shape=(window_size, num_features))
    
    x = layers.LSTM(100, return_sequences=True)(inputs)
    x = layers.LSTM(100)(x)
    
    x = layers.Dense(output_days * output_features)(x)  # Output for 7 days * 2 features (min and max)
    
    x = layers.Reshape((output_days, output_features))(x)
    
    model = models.Model(inputs, x)
    function_name = baseline_LSTM_model.__name__
    
    return model, function_name

In [56]:
def baseline_GRU_model(window_size, num_features, output_days, output_features):
    inputs = layers.Input(shape=(window_size, num_features))
    
    x = layers.GRU(100, return_sequences=False)(inputs)
    
    x = layers.Dense(output_days * output_features)(x)  # Output for 7 days * 2 features (min and max)
    
    x = layers.Reshape((output_days, output_features))(x)
    
    model = models.Model(inputs, x)
    function_name = baseline_GRU_model.__name__
    
    return model, function_name

In [57]:
'''
CNN_LSTM mdoel is a combination of CNN and LSTM. CNN is used to extract the features from the input data and LSTM is used to support the sequence data.
'''
def CNN_LSTM_model(window_size, num_features, output_days, output_features):
    inputs = layers.Input(shape=(window_size, num_features))
    
    x = layers.Conv1D(filters=64, kernel_size=2, activation='relu')(inputs)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=128, kernel_size=2, activation='relu')(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    
    x = layers.LSTM(100, return_sequences=True)(x)
    x = layers.LSTM(100)(x)
    
    x = layers.Dense(output_days * output_features)(x)  # 7 days * 2 targets (High, Low)
    
    x = layers.Reshape((output_days, output_features))(x)
    
    model = models.Model(inputs, x)
    function_name = CNN_LSTM_model.__name__
    
    return model, function_name
#can use KAN, Attenion layer to adjust the weight

In [58]:
# Define the CNN + GRU model
def CNN_GRU_model(window_size, num_features, output_days, output_features):
    inputs = layers.Input(shape=(window_size, num_features))
    
    x = layers.Conv1D(filters=64, kernel_size=2, activation='relu')(inputs)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=128, kernel_size=2, activation='relu')(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    
    x = layers.GRU(100, return_sequences=False)(x)
    
    x = layers.Dense(output_days * output_features)(x)  # Output for 7 days * 2 features (min and max)
    
    x = layers.Reshape((output_days, output_features))(x)
    
    model = models.Model(inputs, x)
    function_name = CNN_GRU_model.__name__
    
    return model, function_name

In [59]:
from tensorflow.keras.layers import Attention
def CNN_LSTM_SA_model(window_size, num_features, output_days, output_features):
    inputs = layers.Input(shape=(window_size, num_features))
    x = layers.Conv1D(filters=64, kernel_size=2, activation='relu')(inputs)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=128, kernel_size=2, activation='relu')(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.LSTM(100, return_sequences=True)(x)
    x = layers.LSTM(100, return_sequences=True)(x)
    
    # Attention mechanism
    attention = Attention()([x, x])
    x = layers.Concatenate()([x, attention])
    x = layers.GlobalAveragePooling1D()(x)
    
    x = layers.Dense(output_days*output_features)(x)
    outputs = layers.Reshape((output_days, output_features))(x)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    function_name = CNN_LSTM_SA_model.__name__
    return model, function_name

In [60]:
def CNN_GRU_SA_model(window_size, num_features, output_days, output_features):
    inputs = layers.Input(shape=(window_size, num_features))
    
    x = layers.Conv1D(filters=64, kernel_size=2, activation='relu')(inputs)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Conv1D(filters=128, kernel_size=2, activation='relu')(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    
    x = layers.GRU(100, return_sequences=True)(x)
    
    attention = layers.Attention()([x, x])  # Self-attention (query = value = x)
    x = layers.Concatenate()([x, attention])
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(output_days * output_features)(x)
    
    outputs = layers.Reshape((output_days, output_features))(x)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    function_name = CNN_GRU_SA_model.__name__
    
    return model,function_name

Meta Model training 

In [139]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.models import save_model

def base_training(model_function,
                loading_path, 
                saving_path, 
                version_name, 
                seed = 42,
                test_size = 0.8,
                epochs = 20,
                batch_size = 8,
                shuffle=True, 
                metrics=['mae'],
                loss = 'mean_squared_error',
                #loss = custom_loss,
                optimizer='adam',
                window_size = window_size, 
                num_features = num_features, 
                output_days = output_days, 
                output_features = output_features):
    
    model_history = []
    
    #create folder to save the model
    model_folder_path = os.path.join(saving_path, version_name, f"_s-{seed}_t-{test_size}_e-{epochs}_b-{batch_size}_S-{shuffle}_m-{metrics}_l-{loss}_o-{optimizer}")

    os.makedirs(model_folder_path, exist_ok=True)
    print(f"Folder '{version_name}' is ready.")


    for window in window_size:
        # Load the data
        X = pd.read_pickle(loading_path + f"X_{window}days_i.pkl")
        y = pd.read_pickle(loading_path + f"y_{window}days_i.pkl")
        print(f"Data for {window} days loaded.")

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, 
                                                            y, 
                                                            test_size=test_size, 
                                                            shuffle=shuffle, 
                                                            random_state=seed)
        print("Data split into training and testing sets.")

        # Train the model
        print("start training:\n")
        model, name = model_function(window, 
                                     num_features, 
                                     output_days, 
                                     output_features)
        model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
        
        history = model.fit(X_train, 
                            y_train, 
                            epochs=epochs, 
                            batch_size=batch_size, 
                            validation_data=(X_test, y_test), 
                            verbose=1)
        
        # Save the model    
        model_save_path = os.path.join(model_folder_path, f"{name}_{window}days.h5")
        model.save(model_save_path)
        print(f"Model trained on {window} days has been saved.")

        model_history.append(history)

    return model_history
    

In [62]:
import matplotlib.pyplot as plt
def plot_training_history(mdoel_history, window=window_size):   
    for i in range(len(mdoel_history)):
        plt.title(f"Training and Validation Loss for {window[i]} days")
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.plot(mdoel_history[i].history['loss'], label='Training Loss')
        plt.plot(mdoel_history[i].history['val_loss'], label='Validation Loss')
        plt.title('Loss over Epochs')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        # Plot MAE
        plt.title(f"Training and Validation MAE for {window[i]} days")
        plt.subplot(1, 2, 2)
        plt.plot(mdoel_history[i].history['mae'], label='Training MAE')
        plt.plot(mdoel_history[i].history['val_mae'], label='Validation MAE')
        plt.title('MAE over Epochs')
        plt.xlabel('Epochs')
        plt.ylabel('MAE')
        plt.legend()
        plt.tight_layout()
        plt.show()


In [138]:
function_list = [baseline_LSTM_model, 
                 baseline_GRU_model, 
                 CNN_LSTM_model, 
                 CNN_GRU_model, 
                 CNN_LSTM_SA_model, 
                 CNN_GRU_SA_model]
for model_function in function_list:
    model_history = base_training(model_function, 
                                  loading_path = "/Users/hoyinchui/Desktop/tbot-st-ta/Testing data/GLD_model_testing_data_i_v5_pkl/", 
                                  saving_path="/Users/hoyinchui/Desktop/tbot-st-ta/saved models/",
                                  version_name="20250128"
                                  )
    plot_training_history(model_history)

Folder '20250128' is ready.
Data for 7 days loaded.
Data split into training and testing sets.
start training:

Epoch 1/20


ValueError: Dimensions must be equal, but are 7 and 2 for '{{node compile_loss/partial/mul}} = Mul[T=DT_FLOAT](compile_loss/partial/Const, compile_loss/partial/Square)' with input shapes: [7], [?,0,2].

Build ensemble model with meta mode (transfer learning/fine-tuning), since will do back propaggation through the ensemble layer all the way to the individual model, we can use multiple stock training data for the individual model, increase the generalization, and do the tuning with the target data, in the ensemble layer training part

In [79]:
from tensorflow.keras.models import Model
def ensemble_stacking (models_inputs, models_outputs, output_days, output_features, optimizer, loss, metrics):
    merged_output = layers.concatenate(models_outputs, axis=-1)
    #searching method for removing the last layer of the model, and directly inputting weight into stackinng model
    
    #stacking model
    merged_output = layers.Dense(64, activation='relu')(merged_output)
    merged_output = layers.Dense(output_days * output_features)(merged_output)
    final_output = layers.Reshape((output_days, output_features))(merged_output)
    ensemble_model = Model(inputs=models_inputs, outputs=final_output)
    ensemble_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return ensemble_model


    

In [85]:
def ensemble_weighting (models_inputs, models_outputs, output_days, output_features, optimizer, loss, metrics):
    merged_output = layers.Add()(models_outputs)
    merged_output = layers.Dense(output_days*output_features)(merged_output)
    final_output = layers.Reshape((output_days, output_features))(merged_output)
    ensemble_model = Model(inputs=models_inputs, outputs=final_output)
    ensemble_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return ensemble_model


In [96]:
#need more understanding on the MoE
''' stacking is y_p = sum(w*y_i)
    MoE is y_p = sum(g_i*y_i)
    can add other function model, like anomaly detection, to the ensemble model, since most of the data is normal, the model can be used to detect the anomaly data(Event)
'''

def ensemble_MoE(models_inputs, models_outputs, output_days, output_features, optimizer, loss, metrics):
    # Step 1: Concatenate the outputs of all the experts
    concatenated_experts = layers.concatenate(models_outputs)
    
    # Step 2: Apply a gate to weight the expert outputs
    gate = layers.Dense(len(models_outputs), activation='softmax')(concatenated_experts)
    
    # Step 3: Multiply the gate with the expert outputs
    weighted_experts = [layers.Multiply()([gate[:, i:i+1], models_outputs[i]]) for i in range(len(models_outputs))]
    
    # Step 4: Sum the weighted expert outputs
    final_expert_output = layers.Add()(weighted_experts)
    
    # Step 5: Reshape the final output to the desired shape
    final_output = layers.Reshape((output_days, output_features))(final_expert_output)

    # Step 6: Define the ensemble model
    ensemble_model = models.Model(inputs=models_inputs, outputs=final_output)

    # Step 7: Compile the model
    ensemble_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return ensemble_model

In [114]:
#function to load models in provided path
from tensorflow.keras.models import load_model
def load_models(path, remove_last_layer=True):
    orginal_models = []
    models_outputs = []
    models_inputs = []
    models_order = []
    for file in sorted(os.listdir(path)):
        if file.endswith(".h5"):
            model = load_model(os.path.join(path, file))
            models_order.append(file)
            print(f"Model loaded: {os.path.join(path, file)}")
            orginal_models.append(load_model(os.path.join(path, file), compile=False))
            if remove_last_layer:
                try:
                    output = model.layers[-2].output  # Use the second last layer as output
                    model = Model(inputs=model.input, outputs=output)
                    print(f"Removed final layer from model: {os.path.join(path, file)}")
                except Exception as e:
                    print(f"Error modifying model {os.path.join(path, file)}: {e}")
            models_inputs.append(model.input)
            print(models_inputs)
            models_outputs.append(model.output)
    
    
    return orginal_models, models_inputs, models_outputs, models_order



In [68]:
def ensemble_model_training(models_inputs, models_outputs, output_days, output_features, model_type = "stacking", optimizer='adam', loss='mean_squared_error', metrics=['mae']):
    if model_type == 'stacking':
        ensemble_model = ensemble_stacking(models_inputs, models_outputs, output_days, output_features, optimizer, loss, metrics)
    elif model_type == 'weighting':
        ensemble_model = ensemble_weighting(models_inputs, models_outputs, output_days, output_features, optimizer, loss, metrics)
    elif model_type == 'MoE':
        ensemble_model = ensemble_MoE(models_inputs, models_outputs, output_days, output_features, optimizer, loss, metrics)
    else:
        print("Please provide a valid model type.")
        return None
    return ensemble_model

In [115]:
#load the whole folder of models (you might have multiple floders of models, then need multiple load_models function and do the selection in the next cell)
orginal_models, models_inputs, models_outputs, models_order = load_models("/Users/hoyinchui/Desktop/tbot-st-ta/saved models/20250128/_s-42_t-0.8_e-20_b-8_S-True_m-['mae']_l-mean_squared_error_o-adam")

Model loaded: /Users/hoyinchui/Desktop/tbot-st-ta/saved models/20250128/_s-42_t-0.8_e-20_b-8_S-True_m-['mae']_l-mean_squared_error_o-adam/CNN_GRU_SA_model_14days.h5
Removed final layer from model: /Users/hoyinchui/Desktop/tbot-st-ta/saved models/20250128/_s-42_t-0.8_e-20_b-8_S-True_m-['mae']_l-mean_squared_error_o-adam/CNN_GRU_SA_model_14days.h5
[<KerasTensor shape=(None, 14, 8), dtype=float32, sparse=False, name=input_layer_45>]
Model loaded: /Users/hoyinchui/Desktop/tbot-st-ta/saved models/20250128/_s-42_t-0.8_e-20_b-8_S-True_m-['mae']_l-mean_squared_error_o-adam/CNN_GRU_SA_model_30days.h5
Removed final layer from model: /Users/hoyinchui/Desktop/tbot-st-ta/saved models/20250128/_s-42_t-0.8_e-20_b-8_S-True_m-['mae']_l-mean_squared_error_o-adam/CNN_GRU_SA_model_30days.h5
[<KerasTensor shape=(None, 14, 8), dtype=float32, sparse=False, name=input_layer_45>, <KerasTensor shape=(None, 30, 8), dtype=float32, sparse=False, name=input_layer_46>]
Model loaded: /Users/hoyinchui/Desktop/tbot-st-

Removed final layer from model: /Users/hoyinchui/Desktop/tbot-st-ta/saved models/20250128/_s-42_t-0.8_e-20_b-8_S-True_m-['mae']_l-mean_squared_error_o-adam/CNN_GRU_model_14days.h5
[<KerasTensor shape=(None, 14, 8), dtype=float32, sparse=False, name=input_layer_45>, <KerasTensor shape=(None, 30, 8), dtype=float32, sparse=False, name=input_layer_46>, <KerasTensor shape=(None, 60, 8), dtype=float32, sparse=False, name=input_layer_47>, <KerasTensor shape=(None, 7, 8), dtype=float32, sparse=False, name=input_layer_44>, <KerasTensor shape=(None, 14, 8), dtype=float32, sparse=False, name=input_layer_37>]
Model loaded: /Users/hoyinchui/Desktop/tbot-st-ta/saved models/20250128/_s-42_t-0.8_e-20_b-8_S-True_m-['mae']_l-mean_squared_error_o-adam/CNN_GRU_model_30days.h5
Removed final layer from model: /Users/hoyinchui/Desktop/tbot-st-ta/saved models/20250128/_s-42_t-0.8_e-20_b-8_S-True_m-['mae']_l-mean_squared_error_o-adam/CNN_GRU_model_30days.h5
[<KerasTensor shape=(None, 14, 8), dtype=float32, spa

Removed final layer from model: /Users/hoyinchui/Desktop/tbot-st-ta/saved models/20250128/_s-42_t-0.8_e-20_b-8_S-True_m-['mae']_l-mean_squared_error_o-adam/CNN_LSTM_SA_model_14days.h5
[<KerasTensor shape=(None, 14, 8), dtype=float32, sparse=False, name=input_layer_45>, <KerasTensor shape=(None, 30, 8), dtype=float32, sparse=False, name=input_layer_46>, <KerasTensor shape=(None, 60, 8), dtype=float32, sparse=False, name=input_layer_47>, <KerasTensor shape=(None, 7, 8), dtype=float32, sparse=False, name=input_layer_44>, <KerasTensor shape=(None, 14, 8), dtype=float32, sparse=False, name=input_layer_37>, <KerasTensor shape=(None, 30, 8), dtype=float32, sparse=False, name=input_layer_38>, <KerasTensor shape=(None, 60, 8), dtype=float32, sparse=False, name=input_layer_39>, <KerasTensor shape=(None, 7, 8), dtype=float32, sparse=False, name=input_layer_36>, <KerasTensor shape=(None, 14, 8), dtype=float32, sparse=False, name=input_layer_41>]
Model loaded: /Users/hoyinchui/Desktop/tbot-st-ta/sa

Model loaded: /Users/hoyinchui/Desktop/tbot-st-ta/saved models/20250128/_s-42_t-0.8_e-20_b-8_S-True_m-['mae']_l-mean_squared_error_o-adam/CNN_LSTM_SA_model_7days.h5
Removed final layer from model: /Users/hoyinchui/Desktop/tbot-st-ta/saved models/20250128/_s-42_t-0.8_e-20_b-8_S-True_m-['mae']_l-mean_squared_error_o-adam/CNN_LSTM_SA_model_7days.h5
[<KerasTensor shape=(None, 14, 8), dtype=float32, sparse=False, name=input_layer_45>, <KerasTensor shape=(None, 30, 8), dtype=float32, sparse=False, name=input_layer_46>, <KerasTensor shape=(None, 60, 8), dtype=float32, sparse=False, name=input_layer_47>, <KerasTensor shape=(None, 7, 8), dtype=float32, sparse=False, name=input_layer_44>, <KerasTensor shape=(None, 14, 8), dtype=float32, sparse=False, name=input_layer_37>, <KerasTensor shape=(None, 30, 8), dtype=float32, sparse=False, name=input_layer_38>, <KerasTensor shape=(None, 60, 8), dtype=float32, sparse=False, name=input_layer_39>, <KerasTensor shape=(None, 7, 8), dtype=float32, sparse=Fa

Removed final layer from model: /Users/hoyinchui/Desktop/tbot-st-ta/saved models/20250128/_s-42_t-0.8_e-20_b-8_S-True_m-['mae']_l-mean_squared_error_o-adam/CNN_LSTM_model_60days.h5
[<KerasTensor shape=(None, 14, 8), dtype=float32, sparse=False, name=input_layer_45>, <KerasTensor shape=(None, 30, 8), dtype=float32, sparse=False, name=input_layer_46>, <KerasTensor shape=(None, 60, 8), dtype=float32, sparse=False, name=input_layer_47>, <KerasTensor shape=(None, 7, 8), dtype=float32, sparse=False, name=input_layer_44>, <KerasTensor shape=(None, 14, 8), dtype=float32, sparse=False, name=input_layer_37>, <KerasTensor shape=(None, 30, 8), dtype=float32, sparse=False, name=input_layer_38>, <KerasTensor shape=(None, 60, 8), dtype=float32, sparse=False, name=input_layer_39>, <KerasTensor shape=(None, 7, 8), dtype=float32, sparse=False, name=input_layer_36>, <KerasTensor shape=(None, 14, 8), dtype=float32, sparse=False, name=input_layer_41>, <KerasTensor shape=(None, 30, 8), dtype=float32, sparse

Model loaded: /Users/hoyinchui/Desktop/tbot-st-ta/saved models/20250128/_s-42_t-0.8_e-20_b-8_S-True_m-['mae']_l-mean_squared_error_o-adam/baseline_LSTM_model_14days.h5
Removed final layer from model: /Users/hoyinchui/Desktop/tbot-st-ta/saved models/20250128/_s-42_t-0.8_e-20_b-8_S-True_m-['mae']_l-mean_squared_error_o-adam/baseline_LSTM_model_14days.h5
[<KerasTensor shape=(None, 14, 8), dtype=float32, sparse=False, name=input_layer_45>, <KerasTensor shape=(None, 30, 8), dtype=float32, sparse=False, name=input_layer_46>, <KerasTensor shape=(None, 60, 8), dtype=float32, sparse=False, name=input_layer_47>, <KerasTensor shape=(None, 7, 8), dtype=float32, sparse=False, name=input_layer_44>, <KerasTensor shape=(None, 14, 8), dtype=float32, sparse=False, name=input_layer_37>, <KerasTensor shape=(None, 30, 8), dtype=float32, sparse=False, name=input_layer_38>, <KerasTensor shape=(None, 60, 8), dtype=float32, sparse=False, name=input_layer_39>, <KerasTensor shape=(None, 7, 8), dtype=float32, spa

In [117]:
#select the models(4 models Time-based) and ensemble method you want and feed into the ensemble traing function
#if stargetry changed, the function and training set need to updatye too 
print(models_order)

['CNN_GRU_SA_model_14days.h5', 'CNN_GRU_SA_model_30days.h5', 'CNN_GRU_SA_model_60days.h5', 'CNN_GRU_SA_model_7days.h5', 'CNN_GRU_model_14days.h5', 'CNN_GRU_model_30days.h5', 'CNN_GRU_model_60days.h5', 'CNN_GRU_model_7days.h5', 'CNN_LSTM_SA_model_14days.h5', 'CNN_LSTM_SA_model_30days.h5', 'CNN_LSTM_SA_model_60days.h5', 'CNN_LSTM_SA_model_7days.h5', 'CNN_LSTM_model_14days.h5', 'CNN_LSTM_model_30days.h5', 'CNN_LSTM_model_60days.h5', 'CNN_LSTM_model_7days.h5', 'baseline_GRU_model_14days.h5', 'baseline_GRU_model_30days.h5', 'baseline_GRU_model_60days.h5', 'baseline_GRU_model_7days.h5', 'baseline_LSTM_model_14days.h5', 'baseline_LSTM_model_30days.h5', 'baseline_LSTM_model_60days.h5', 'baseline_LSTM_model_7days.h5']


In [131]:
#be aware of the order, it need to be 7, 14, 30, 60 from small to large
models_inputs_selected = [models_inputs[3], models_inputs[0], models_inputs[1], models_inputs[2]]   
models_outputs_selected = [models_outputs[3], models_outputs[0], models_outputs[1], models_outputs[2]]

In [127]:
#select the models(4 models Time-based) and ensemble method you want and feed into the ensemble traing function
#if stargetry changed, the function and training set need to updatye too 
ensemble_s_model = ensemble_model_training(models_inputs_selected, models_outputs_selected, output_days, output_features, model_type = "stacking", optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [128]:
ensemble_w_model = ensemble_model_training(models_inputs_selected, models_outputs_selected, output_days, output_features, model_type = "weighting", optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [129]:
ensemble_MoE_model = ensemble_model_training(models_inputs_selected, models_outputs_selected, output_days, output_features, model_type = "MoE", optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [100]:
#loading the data for testing in ensemble model

def load_data(window_size = window_size, path = f"/Users/hoyinchui/Desktop/tbot-st-ta/Testing data/GLD_model_testing_data_pre_ens_Ai1_v3_pkl", test_size = 0.8, shuffle = True, seed = 42):
    all_X_train = []
    all_X_test = []
    all_y_train = []
    all_y_test = []
    
    y = pd.read_pickle(f"{path}/y_a.pkl")
    for window in window_size:
        X = pd.read_pickle(f"{path}/X_{window}days_a.pkl")
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=shuffle, random_state=seed)
        all_X_train.append(X_train)
        all_X_test.append(X_test)
        all_y_train.append(y_train)
        all_y_test.append(y_test)
    return all_X_train, all_X_test, all_y_train, all_y_test

In [119]:
all_X_train, all_X_test, all_y_train, all_y_test = load_data(path = f"/Users/hoyinchui/Desktop/tbot-st-ta/Testing data/GLD_model_testing_data_pre_ens_Ai1_v3_pkl", test_size = 0.8, shuffle = True, seed = 42)

In [122]:
#fit the ensemble model
def ensemble_model_fit(ensemble_model, all_X_train, all_y_train, epochs = 20, batch_size = 8):
    history = ensemble_model.fit(all_X_train, all_y_train[0], epochs=epochs, batch_size=batch_size, verbose=1)
    return history

In [ ]:
#traing the ensemble model
ensemble_s_model_history = ensemble_model_fit(ensemble_s_model, all_X_train, all_y_train, epochs = 20, batch_size = 8)

Epoch 1/20


/opt/anaconda3/lib/python3.12/site-packages/keras/src/ops/nn.py:907: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


126/126 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - loss: 0.0307 - mae: 0.0906
Epoch 2/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 4.9231e-04 - mae: 0.0163
Epoch 3/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 4.7505e-04 - mae: 0.0162
Epoch 4/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 5.6417e-04 - mae: 0.0175
Epoch 5/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 4.4949e-04 - mae: 0.0159
Epoch 6/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 4.1725e-04 - mae: 0.0152
Epoch 7/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 5.3108e-04 - mae: 0.0167
Epoch 8/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 3.7259e-04 - mae: 0.0144
Epoch 9/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 4.1055e-04 - mae: 0.0150
Epoch 10/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 4.5056e-04 - mae: 0.0158
Epoch 11/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 3.7371e-04 - mae: 0.0142
Epoch 12/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 3.306

### the following 3 cell can skip, theey are old version for reference

In [ ]:
#load the model and predict and save the prediction
#for window in window_size:
#    model = tf.keras.models.load_model(f"CNN_LSTM_{window}days.h5")
#    y_pred = model.predict(X_test)
#    pd.DataFrame(y_pred).to_csv(f"y_pred_CNN_LSTM_{window}days.csv")

In [ ]:
#putting all windows prediction into the dataframe as X
#import numpy as np
#X_ensemble = pd.DataFrame()
#for window in window_size:
#    X_ensemble = pd.concat([X_ensemble, pd.read_csv(f"y_pred_CNN_LSTM_{window}days.csv")], axis=1)
#    #combein the columns, so that can be used as input for the ensemble model
#   X_ensemble = X_ensemble.applymap(lambda x: np.vstack(x))

They can combein in 1 for loop, I just split it for clear

In [ ]:
#def train_test_split_ensemble(X,y,test_size=0.5, seed=42):    
#    #split again for the prediction model and the ensemble model
#    #Since we already shuffled the data, we can just split the data in half, and easaier to manage
#    X_train_ensemble, X_test_ensemble, y_train_ensemble, y_test_ensemble = train_test_split(X, y, test_size=test_size, shuffle=False, random_state=seed)
#    return X_train_ensemble, X_test_ensemble, y_train_ensemble, y_test_ensemble

In [ ]:
#X_train_ensemble, X_test_ensemble, y_train_ensemble, y_test_ensemble = train_test_split_ensemble(X_ensemble, y_test)

### Ensemble Model (TBC)

In [3]:
#from tensorflow.keras.models import Model
#def ensemble_CNN_LSTM(window_size, num_features):
#    inputs = layers.Input(shape=(window_size, num_features))
#    x = layers.Conv1D(filters=64, kernel_size=2, activation='relu')(inputs)
#    x = layers.MaxPooling1D(pool_size=2)(x)
#    x = layers.Conv1D(filters=128, kernel_size=2, activation='relu')(x)
#    x = layers.MaxPooling1D(pool_size=2)(x)
#    x = layers.LSTM(100)(x)
#    model = Model(inputs=inputs, outputs=x)
#    return model

In [ ]:
#def ensemble_s_models(window_size, num_features, output_days, output_features):
#        models = []
#    for window in window_size:
#        model = ensemble_CNN_LSTM(window, num_features)
#        models.append(model)

In [ ]:
#def ensemble_MoE_models(window_size, num_features, output_days, output_features):


In [4]:
#def ensemble_h_models(window_size, num_features, output_days, output_features):
#    models = []
#    for window in window_size:
#        model = ensemble_CNN_LSTM(window, num_features)
#        models.append(model)
#    models_inputs = [model.input for model in models]
#    models_outputs = [model.output for model in models]
#   merged = layers.concatenate(models_outputs, axis=-1)
#    merged_output = layers.Dense(output_days * output_features)(merged)
#    final_output = layers.Reshape((output_days, output_features))(merged_output)
#    ensemble_model = Model(inputs=models_inputs, outputs=final_output)
#    ensemble_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
#
#    return ensemble_model

In [6]:
#from sklearn.model_selection import train_test_split
#import pandas as pd
#seed = 42
#test_size = 0.8 # we need to consider for the ensemble model's training and ttesting data, since it cannot use the same training data
#epochs = 20
#batch_size = 8
#shuffle=True
#all_X_train = []
#all_X_test = []
#all_y_train = []
#all_y_test = []
#
#y = pd.read_pickle(f"/Users/hoyinchui/Downloads/y_a.pkl")
#for window in window_size:
#    X = pd.read_pickle(f"/Users/hoyinchui/Downloads/X_{window}days_a.pkl")
#    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=shuffle, random_state=seed)
#    all_X_train.append(X_train)
#    all_X_test.append(X_test)
#    all_y_train.append(y_train)
#    all_y_test.append(y_test)
#
##ensemble_model = ensemble_h_models(window_size, num_features, output_days, output_features)
##history = ensemble_model.fit(all_X_train, all_y_train[0], epochs=epochs, batch_size=batch_size, validation_data=(all_X_test, all_y_test[0]), verbose=1)
##ensemble_model.save(f"ensemble_CNN_LSTM.h5")
#

Epoch 1/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 0.0279 - mae: 0.1027 - val_loss: 5.5140e-04 - val_mae: 0.0176
Epoch 2/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 5.2824e-04 - mae: 0.0169 - val_loss: 4.1390e-04 - val_mae: 0.0154
Epoch 3/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 5.1239e-04 - mae: 0.0167 - val_loss: 4.6971e-04 - val_mae: 0.0162
Epoch 4/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 4.3028e-04 - mae: 0.0155 - val_loss: 0.0011 - val_mae: 0.0263
Epoch 5/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 7.6607e-04 - mae: 0.0210 - val_loss: 4.8477e-04 - val_mae: 0.0170
Epoch 6/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 4.1826e-04 - mae: 0.0152 - val_loss: 3.3729e-04 - val_mae: 0.0135
Epoch 7/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 3.9106e-04 - mae: 0.0147 - val_loss: 5.9622e-04 - val_mae: 0.0190
Epoch 8/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 3.8556e-04 - mae: 0.0144 - val_loss: 3.2576e-04 - val_mae: